In [1]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datasets import load_dataset
from pymatgen.core import Structure
from pymatgen.util.testing import PymatgenTest
from scipy.spatial.distance import jensenshannon

from lematerial_forgebench.benchmarks.distribution_benchmark import (
    DistributionBenchmark,
)
from lematerial_forgebench.metrics.distribution_metrics import (
    MMD,
    FrechetDistance,
    JSDistance,
)
from lematerial_forgebench.preprocess.distribution_preprocess import (
    DistributionPreprocessor,
)
from lematerial_forgebench.utils.distribution_utils import (
    map_space_group_to_crystal_system,
)

%load_ext autoreload
%autoreload 2

In [2]:
with open("small_lematbulk.pkl", "rb") as f:
    test_lemat = pickle.load(f)

In [3]:
test = PymatgenTest()

filename = "CsBr.cif"
structure = Structure.from_file(filename)
structure = structure.remove_oxidation_states()

filename2 = "CsPbBr3.cif"
structure2 = Structure.from_file(filename2)
structure2 = structure2.remove_oxidation_states()

structures = [
    structure,
    structure2,
    test.get_structure("Si"),
    test.get_structure("LiFePO4"),
]

C:\Users\samue\AppData\Local\Temp\ipykernel_11736\2264363178.py:1: FutureWarning: PymatgenTest is deprecated, and will be removed on 2026-01-01
Use MatSciTest in pymatgen.util.testing instead.
  test = PymatgenTest()


In [4]:
distribution_preprocessor = DistributionPreprocessor()
preprocessor_result = distribution_preprocessor(structures)

In [5]:
preprocessor_result.processed_structures[0].properties.get("distribution_properties")

{'Volume': 392.55602810984936,
 'Density(g/cm^3)': 3.600794626101272,
 'Density(atoms/A^3)': 0.020379256531914348,
 'SpaceGroup': 225,
 'CrystalSystem': 7,
 'CompositionCounts': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Composition': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.

In [8]:
metric = FrechetDistance(reference_df=test_lemat)

[autoreload of lematerial_forgebench.metrics.distribution_metrics failed: Traceback (most recent call last):
  File "C:\Users\samue\AppData\Local\uv\cache\archive-v0\Lj7vvSFvlp-U62Tn9J8TQ\Lib\site-packages\IPython\extensions\autoreload.py", line 280, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\samue\AppData\Local\uv\cache\archive-v0\Lj7vvSFvlp-U62Tn9J8TQ\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 533, in maybe_reload_module
    new_source_code = f.read()
                      ^^^^^^^^
  File "C:\Users\samue\AppData\Roaming\uv\python\cpython-3.11.11-windows-x86_64-none\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 290: character maps to <undefined>
]


In [7]:
metric.compute(preprocessor_result.processed_structures)

       Volume  Density(g/cm^3)  Density(atoms/A^3)  SpaceGroup  CrystalSystem  \
0  392.556028         3.600795            0.020379         225              7   
1  769.207176         5.006763            0.026001          62              3   
2   40.044795         2.329245            0.049944         227              7   
3  299.607968         3.497400            0.093455          14              2   

                                   CompositionCounts  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0,...   

                                         Composition  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...  


NameError: name 'reference_df' is not defined

In [8]:
benchmark = DistributionBenchmark(test_lemat)

In [11]:
benchmark_result = benchmark.evaluate([test_df])

[13:42:23] WARNING  Failed to compute metric for structure 0                                            ]8;id=658320;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\base.py\base.py]8;;\:]8;id=763406;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\base.py#349\349]8;;\
                    ╭─────────────────────── Traceback (most recent call last) ───────────────────────╮            
                    │ C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\base.py: │            
                    │ 341 in compute                                                                  │            
                    │                                                                                 │            
                    │   338 │   │   │   │   # Serial computation                                      │            
                    │   339 │   │   │   │   for idx, structure in enumerate(structures):              │            
                    │   340 │   │   │   │   │   try:                                                  │            
                    │ ❱ 341 │   │   │   │   │   │   value = self.compute_structure(structure, **compu │            
                    │   342 │   │   │   │   │   │   values.append(value)                              │            
                    │   343 │   │   │   │   │   except Exception as e:                                │            
                    │   344 │   │   │   │   │   │   failed_indices.append(idx)                        │            
                    │                                                                                 │            
                    │ C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\distribu │            
                    │ tion_metrics.py:134 in compute_structure                                        │            
                    │                                                                                 │            
                    │   131 │   │   │   │   │   dist_metrics[quant] = js                              │            
                    │   132 │   │                                                                     │            
                    │   133 │   │   for quant in ["CompositionCounts", "Composition"]:                │            
                    │ ❱ 134 │   │   │   js = compute_jensen_shannon_distance(                         │            
                    │   135 │   │   │   │   reference_df,                                             │            
                    │   136 │   │   │   │   structure,                                                │            
                    │   137 │   │   │   │   quant,                                                    │            
                    │                                                                                 │            
                    │ C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\utils\distributi │            
                    │ on_utils.py:118 in compute_jensen_shannon_distance                              │            
                    │                                                                                 │            
                    │   115 │   │   )                                                                 │            
                    │   116 │                                                                         │            
                    │   117 │   elif crystal_param == "CompositionCounts":                            │            
                    │ ❱ 118 │   │   with open("data/lematbulk_composition_counts_distribution.json",  │            
                    │   119 │   │   │   reference_data_dist = json.load(file)                         │            
                    │   120 │   elif crystal_param == "Composition":

           WARNING  Failed to compute metric for structure 0                                            ]8;id=614259;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\base.py\base.py]8;;\:]8;id=169555;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\base.py#349\349]8;;\
                    ╭─────────────────────── Traceback (most recent call last) ───────────────────────╮            
                    │ C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\base.py: │            
                    │ 341 in compute                                                                  │            
                    │                                                                                 │            
                    │   338 │   │   │   │   # Serial computation                                      │            
                    │   339 │   │   │   │   for idx, structure in enumerate(structures):              │            
                    │   340 │   │   │   │   │   try:                                                  │            
                    │ ❱ 341 │   │   │   │   │   │   value = self.compute_structure(structure, **compu │            
                    │   342 │   │   │   │   │   │   values.append(value)                              │            
                    │   343 │   │   │   │   │   except Exception as e:                                │            
                    │   344 │   │   │   │   │   │   failed_indices.append(idx)                        │            
                    │                                                                                 │            
                    │ C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\distribu │            
                    │ tion_metrics.py:345 in compute_structure                                        │            
                    │                                                                                 │            
                    │   342 │   │   quantities = structure.columns                                    │            
                    │   343 │   │   for quant in quantities:                                          │            
                    │   344 │   │   │   if quant in reference_df.columns:                             │            
                    │ ❱ 345 │   │   │   │   frechetdist = compute_frechetdist(reference_df, structure │            
                    │   346 │   │   │   │   dist_metrics[quant] = frechetdist                         │            
                    │   347 │   │                                                                     │            
                    │   348 │   │   return dist_metrics                                               │            
                    │                                                                                 │            
                    │ C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\utils\distributi │            
                    │ on_utils.py:156 in compute_frechetdist                                          │            
                    │                                                                                 │            
                    │   153 │   │   reference_data, metric=crystal_param, return_2d_array=True        │            
                    │   154 │   )                                                                     │            
                    │   155 │                                                                         │            
                    │ ❱ 156 │   distance = frdist(reference_data_dist, generated_crystals_dist)       │            
                    │   157 │   return distance                                                       │            
                    │   158                                         

MMD_results


In [215]:
benchmark_result.evaluator_results["JSDistance"]["JSDistance_value"]

{'SpaceGroup': np.float64(0.8325546111576977),
 'CrystalSystem': np.float64(0.2788095948658411),
 'CompositionCounts': np.float64(0.6901659684588751),
 'Composition': np.float64(0.713320308611451)}

In [216]:
benchmark_result.evaluator_results["MMD"]["MMD_value"]

{'Volume': np.float64(0.3578373686257075),
 'Density(g/cm^3)': np.float64(0.16174021104526248),
 'Density(atoms/A^3)': np.float64(0.0003819190830129937)}